In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import datetime
import random
import re
import pprint
import json

def getMatchLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk/rugby/fixtures/_/date/" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                      t.string, flags=re.DOTALL | re.MULTILINE).group(1)

    json_out = json.loads(json_text)
    
    # Get just the match links
    json_match = json_out['schedule']['groups']
    matches = []
    for m in json_match:
        for c in m['complete']:
            matches.append(c['result']['href'])
            
    return matches

links = getMatchLinks('20170304')

In [3]:
def getStatsLinks (articleUrl):
    html = urlopen("http://www.espn.co.uk" + articleUrl)
    bsObj = bs(html, "lxml")
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    json_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                          t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        json_out.append(json.loads(json_text))

    for j in json_out[0]['gamePackage']['links']:
        if j['pageType'] == 'matchstats':
            match_s = j['href']
        elif j['pageType'] == 'playerstats':
            player_s = j['href']
            
    return match_s, player_s

m, p = getStatsLinks(links[0])
print(m)
print(p)

/rugby/matchstats?gameId=290088&league=267979
/rugby/playerstats?gameId=290088&league=267979


In [4]:
def getStats(statsUrl):
    html = urlopen("http://www.espn.co.uk" + statsUrl)
    bsObj = bs(html, "lxml")  
    tst = bsObj.find("section", {"id":"pane-main"}).findAll("script", {"type":"text/javascript"})
    
    stat_out = []
    for t in tst:
        json_text = re.search(r'^\s*window\.__INITIAL_STATE__\s*=\s*({.*?})\s*;\s*$',
                              t.string, flags=re.DOTALL | re.MULTILINE).group(1)
        stat_out.append(json.loads(json_text))
        
    return stat_out[0]

In [297]:
tst = getStats('/rugby/matchstats?gameId=290088&league=267979')

for t in tst['gamePackage']:
    print(t)

matchHomeForm
commentaryFeedback
article
news
matchGlossary
polling
matchAttacking
matchDetails
gameState
standings
gameStateClass
headToHead
matchCommentary
matchEvents
matchAwayForm
leagueUid
showGameDetailFooter
matchSummary
meta
matchDefending
HeadToHeadNode
gameStrip
matchStats
matchLineUp
matchDiscipline
analytics
matchConversation
links


In [160]:
from pprint import pprint
# Kicks/passes/runs, metres run, tries/penalties/conversions
pprint(tst['gamePackage']['matchEvents'])
print ('----------------------------------')
# Overall possession/territory, Runs/metres/clean breaks/tackles/missed tackles/turnovers
pprint(tst['gamePackage']['matchStats'])
print ('----------------------------------')
# defenders beaten, poss/terr per half, offloads, rucks won, mauls won
pprint(tst['gamePackage']['matchAttacking'])
print ('----------------------------------')
# Scrums and lineouts, tackles made and missed, 
pprint(tst['gamePackage']['matchDefending'])
print ('----------------------------------')
# Penalties conceded, yellow + red cards, free kicks conceded
pprint(tst['gamePackage']['matchDiscipline'])

{'away': {'color': '#000000',
          'logo': 'http://a1.espncdn.com/combiner/i?img=/i/teamlogos/rugby/teams/500/25905.png&h=35&w=35',
          'name': 'WASPS'},
 'col': [[{'data': [{'awayValue': '3', 'homeValue': '0', 'text': 'Tries'},
                    {'awayValue': '3',
                     'homeValue': '0',
                     'text': 'Conversion Goals'},
                    {'awayValue': '1',
                     'homeValue': '1',
                     'text': 'Penalty Goals'},
                    {'awayValue': '100%',
                     'homeValue': '100%',
                     'text': 'Kick Percent Success'}],
           'type': 'table'}],
         [{'data': {'awayText': '591',
                    'homeText': '381',
                    'homeValue': 39,
                    'text': 'Metres Run'},
           'type': 'dataVis'},
          {'data': [{'awayValue': '34',
                     'homeValue': '32',
                     'text': 'Kicks From Hand'},
                    

In [165]:
for t in tst['gamePackage']['matchEvents']:
    print(t)

home
split
col
title
away


In [202]:
cols = []
vals = []
for t in tst['gamePackage']['matchEvents']['col']:
    if type(t) is list:
        for x in t:
            pprint(x['data'])

[{'awayValue': '3', 'homeValue': '0', 'text': 'Tries'},
 {'awayValue': '3', 'homeValue': '0', 'text': 'Conversion Goals'},
 {'awayValue': '1', 'homeValue': '1', 'text': 'Penalty Goals'},
 {'awayValue': '100%', 'homeValue': '100%', 'text': 'Kick Percent Success'}]
{'awayText': '591', 'homeText': '381', 'homeValue': 39, 'text': 'Metres Run'}
[{'awayValue': '34', 'homeValue': '32', 'text': 'Kicks From Hand'},
 {'awayValue': '173', 'homeValue': '158', 'text': 'Passes'},
 {'awayValue': '133', 'homeValue': '101', 'text': 'Runs'}]


In [298]:
kpr = tst['gamePackage']['matchEvents']['col'][1][1]

pprint(kpr)

{'data': [{'awayValue': '34', 'homeValue': '32', 'text': 'Kicks From Hand'},
          {'awayValue': '173', 'homeValue': '158', 'text': 'Passes'},
          {'awayValue': '133', 'homeValue': '101', 'text': 'Runs'}],
 'type': 'table'}


In [299]:
# Rename the fields in the dictionaries so know what they are when they go into the DB
new_kpr = kpr.copy()
for k in new_kpr['data']:
    if k['text'] == 'Kicks From Hand':
        k['away_kicks_from_hand'] = k.pop('awayValue')
        k['home_kicks_from_hand'] = k.pop('homeValue')
    elif k['text'] == 'Passes':
        k['away_passes'] = k.pop('awayValue')
        k['home_passes'] = k.pop('homeValue')
    elif k['text'] == 'Runs':
        k['away_runs'] = k.pop('awayValue')
        k['home_runs'] = k.pop('homeValue')

In [250]:
pprint(new_kpr)

{'data': [{'away_kicks_from_hand': '34',
           'home_kicks_from_hand': '32',
           'text': 'Kicks From Hand'},
          {'away_passes': '173', 'home_passes': '158', 'text': 'Passes'},
          {'away_runs': '133', 'home_runs': '101', 'text': 'Runs'}],
 'type': 'table'}


In [300]:
pprint(new_kpr['data'][0])

# Add the col name and value to two lists
vals, cols = [], []
for k in new_kpr['data'][0]:
    if k == 'text':
        continue
    else:
        cols.append(k)
        vals.append(new_kpr['data'][0][k])

{'away_kicks_from_hand': '34',
 'home_kicks_from_hand': '32',
 'text': 'Kicks From Hand'}


In [301]:
print(cols, vals)

['away_kicks_from_hand', 'home_kicks_from_hand'] ['34', '32']


In [284]:
poss = tst['gamePackage']['matchStats']['dataVis']
pprint(poss)

[{'awayText': '49%',
  'awayValue': 49,
  'homeText': '50%',
  'homeValue': 50,
  'text': 'Possession'},
 {'awayText': '56%',
  'awayValue': 56,
  'homeText': '43%',
  'homeValue': 43,
  'text': 'Territory'}]


In [295]:
new_poss = poss.copy()
for p in new_poss:
    if p['text'] == 'Possession':
        p['home_possession'] = p.pop('homeValue')
        p['away_possession'] = p.pop('awayValue')
    elif p['text'] == 'Territory':
        p['home_territory'] = p.pop('homeValue')
        p['away_territory'] = p.pop('awayValue')

In [296]:
new_poss

[{'awayText': '49%',
  'away_possession': 49,
  'homeText': '50%',
  'home_possession': 50,
  'text': 'Possession'},
 {'awayText': '56%',
  'away_territory': 56,
  'homeText': '43%',
  'home_territory': 43,
  'text': 'Territory'}]